### Deploy and Interact with LUCE contract
The aim of this notebook is to compile, deploy and interact with all functions of the LUCE smart contract (v3).

### Compile contract from file

In [2]:
# Import libraries
import json
import web3

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

In [3]:
# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

In [4]:
# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

In [5]:
#compiled_sol

In [6]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

In [10]:
#contract_interface

In [8]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

In [13]:
#contract_interface['bin']

### Deploy

In [8]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [9]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [10]:
# Create contract blueprint
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

In [11]:
# Submit the transaction that deploys the contract
tx_hash = Luce.constructor().transact()

### Obtain Transcation Receipt

In [12]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [13]:
# Obtain address of freshly deployed contract
tx_receipt.contractAddress

'0xb88966F5cEc4612495F0B9D42b0b4Ad5B0Cf6329'

### Interact with contract

In [14]:
# Create python instance of deployed contract
luce1 = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

**publishData**  
```solidity
publishData(string memory _newdescription, string memory _link, uint _licence)
```

In [15]:
# Publish Data
tx_hash = luce1.functions.publishData("example description", "my link", 6).transact()

**setLicence**
```solidity
setLicence(uint newLicence)
```

In [16]:
# Set Licence
_ = luce1.functions.setLicence(5).transact()

From here on I am using `_` to suppress the output of the transaction hash. I do this since transactions are mined instantaneously in the local ganache test setting. On the testnet and mainnet we would store the transaction hash, then wait until a transaction receipt is available and only then we can be sure that the transaction went through.

**getLicence**
```solidity
getLicence() public view returns(uint)
```

In [17]:
# Get Licence
luce1.functions.getLicence().call()

5

**updateData**
```solidity
updateData(string memory updateDescr, string memory _newlink)
```

In [29]:
# Update Data
_ = luce1.functions.updateData("New Description", "New Link").transact()

**getLink**
```solidity
getLink(uint token) public view returns(string memory)
```

In [30]:
luce1.functions.getLink(1).call()

'New Link'

All LUCE functions up to this point are working.

---

We can also query the dataProvider and dataDescription since public variables in smart contracts automatically have a getter method. We simply call them by their name to read the current variable state:

**dataProvider**

In [31]:
luce1.functions.dataProvider().call()

'0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA'

**dataDescription**

In [32]:
luce1.functions.dataDescription().call()

'New Description'

---

**UPDATE:**  
All working as expected now

**addDataRequester**
```solidity
addDataRequester(uint purposeCode, uint licenceType) public returns(uint)
```

In [34]:
luce1.functions.addDataRequester(1,5).transact()

HexBytes('0x6c6702b5f9084e804af84fb3544b3b1d6126095b0592e9eeb957ff3018dbc886')

**renewToken**
```solidity
renewToken(uint compliance) public returns(uint token)
```

In [25]:
luce1.functions.renewToken(2).transact()
# Error: Need to agree on licence first

HexBytes('0x2415c718725b570ab73fbf6b9569a7dd76135e6888f23bf2eaa0c9ea501dc054')